ΟΝΟΜΑΤΕΠΩΝΥΜΟ:ΙΟΝ ΑΝΤΟΝΙΟ ΜΑΖΙΛΟΥ

ΑΜ:2022202000131

ΟΝΟΜΑΤΕΠΩΝΥΜΟ:ΠΑΥΛΟΣ ΣΥΓΡΙΜΗΣ

ΑΜ:2022202000202

## Αναζήτηση με Αντιπαλότητα

Σε αυτό το εργαστήριο, θα μελετήσουμε και θα υλοποιήσουμε αλγορίθμους αναζήτησης με αντιπαλότητα. Παραδοσιακά, οι αλγόριθμοι αυτοί εφαρμόζονται σε παιχνίδια όπως το σκάκι και η τρίλιζα. Στα πλαίσια του εργαστηρίου, θα υλοποιήσουμε τον αλγόριθμο Minimax και θα τον χρησιμοποιήσουμε ως τον αντίπαλό μας στο παιχνίδι της τρίλιζας.

### Κατάσταση παιχνιδιού
Για να ανπαραστήσουμε κάθε κατάσταση του παιχνιδιού, θα χρησιμοποιήσουμε ένα namedtuple το οποίο είναι ένας τύπος tuple όπου κάθε θέση του tuple είναι ονοματισμένη. Θα ονομάσουμε το namedtuple που θα ορίσουμε `GameState` και κάθε τέτοιο namedtuple θα αποτελείται από τα παρακάτω 4 στοιχεία:
1. `to_move`: Μας λέει ποιού παίκτη είναι η σειρά να παίξει στη συνέχεια.
2. `utility`: Αποθηκεύει τη χρησιμότητα της κατάστασης (π.χ., την τιμή minimax του αντίστοιχου κόμβου).
3. `board`: Ένα λεξικό που αποθηκεύει το πλέγμα του παιχνιδιού (π.χ., θέσεις των κομματιών στο σκάκι).
4. `moves`: Μια λίστα με τις νόμιμες κινήσεις που είναι δυνατές από την τρέχουσα κατάσταση.

<u>Πράγματα να κάνω:</u>

- Εκτελέστε τον κώδικα στο παρακάτω κελί για να ορίσετε το namedtuple `GameState`.

In [2]:
from collections import namedtuple

GameState = namedtuple('GameState', 'to_move, utility, board, moves')

### Παιχνίδι
Στη συνέχεια, θα ορίσουμε την αφηρημένη κλάση `Game` την οποία μπορούμε να χρησιμοποιήσουμε για να απαραστήσουμε τα παιχνίδια μας.

<u>Πράγματα να κάνω:</u>

- Εκτελέστε τον κώδικα στο παρακάτω κελί για να ορίσετε την κλάση `Game`.

In [3]:
class Game:
    """A game is similar to a problem, but it has a utility for each
    state and a terminal test instead of a path cost and a goal
    test. To create a game, subclass this class and implement actions,
    result, utility, and terminal_test. You may override display and
    successors or you can inherit their default methods. You will also
    need to set the .initial attribute to the initial state; this can
    be done in the constructor."""

    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move

    def display(self, state):
        """Print or otherwise display the state."""
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                if self.terminal_test(state):
                    self.display(state)
                    return self.utility(state, self.to_move(self.initial))

Ας δούμε τώρα μερικές λεπτομέρειες για τις μεθόδους της κλασής `Game`. Πρέπει να υλοποιήσετε αυτές τις μεθόδους όταν δημιουργείτε νέες κλάσεις που κληρονομούν από την `Game` και οι οποίες αναπαριστούν ένα παιχνίδι.

* `actions(self, state)`: Δεδομένης μιας κατάστασης παιχνιδιού `state`, αυτή η μέθοδος επιστρέφει μια λίστα που περιέχει όλες τις νόμιμες ενέργειες που μπορούν να πραγματοποιηθούν από αυτήν την κατάσταση.

* `result(self, state, action)`: Δεδομένης μιας κατάστασης παιχνιδιού `state` και μιας κίνησης `action`, αυτή η μέθοδος επιστρέφει την νέα κατάσταση που προκύπτει αν εφαρμοστεί η συγκεκριμένη κίνηση στην τρέχουσα κατάσταση παιχνιδιού.

* `utility(self, state, player)`: Δεδομένης μιας τελικής κατάστασης του παιχνιδιού `state` και ενός παίκτη `player`, αυτή η μέθοδος επιστρέφει τη χρησιμότητα της τελικής αυτής κατάστασης για τον παίκτη αυτόν.

* `terminal_test(self, state)`: Δεδομένης μιας κατάστασης παιχνιδιού `state`, αυτή η μέθοδος επιστρέφει `True` εάν αυτή η κατάσταση είναι τελική και `False` διαφορετικά.

* `to_move(self, state)`: Δεδομένης μιας κατάστασης παιχνιδιού `state`, αυτή η μέθοδος επιστρέφει τον παίκτη που θα παίξει στη συνέχεια.

* `display(self, state)`: Αυτή η μέθοδος εκτυπώνει/εμφανίζει την τρέχουσα κατάσταση του παιχνιδιού.

### Τρίλιζα
Το επόμενο πράγμα που θα κάνουμε είναι να δημιουργήσουμε μια κλάση η οποία αναπαριστά το παιχνίδι της τρίλιζας. Ωστόσο, πριν δημιουργήσουμε την κλάση, πρέπει να αποφασίσουμε πώς ακριβώς θα αναπαραστήσουμε κάθε κατάσταση της τρίλιζας ως namedtuple `GameState`. Μια τέτοια κατάσταση πρέπει να μας δίνει όλες τις πληροφορίες για το παιχνίδι ανά πάσα στιγμή. Συγκεκριμένα, στις θέσεις `to_move`, `utility`, `board` και `moves` του `GameState`, θα αποθηκεύσουμε τα ακόλουθα:

* `to_move`: Ένα αλφαριθμητικό ενός χαρακτήρα, είτε 'X' ή 'O'.

* `utility`: 1 για νίκη (του παίκτη που έπαιξε τελευταίος), -1 για ήττα, 0 για ισοπαλία.

* `board`: Ένα λεξικό όπου τα κλειδιά είναι οι μη κενές θέσεις του πλέγματος και οι τιμές είναι τα σύμβολα (δηλ. 'X' ή 'O') που έχουν τοποθετηθεί σε αυτές τις θέσεις.

* `moves`: Μια λίστα με όλες τις πιθανές κινήσεις από την τρέχουσα κατάσταση. Κάθε κίνηση είναι ένα tuple που αναπαριστά τις συντεταγμένες ενός μη κατειλημμένου κελιού.


Παρακάτω, θα ορίσουμε την κλάση `TicTacToe` η οποία είναι υποκλάση της κλάσης `Game` και η οποία αναπαριστά το παιχνίδι της τρίλιζας. Η κλάση `TicTacToe` υλοποιεί όλες τις μεθόδους της κλάσης `Game`. Υπάρχουν κάποιες επιπλέον μέθοδοι:

* `__init__(self, h=3, v=3, k=3)`: Η μέθοδος αυτή αποτελεί τον κατασκευαστή της κλάσης. Δημιουργεί ένα αντικείμενο της κλάσης και αρχικοποιεί το παιχνίδι της τρίλιζας. Η αρχικοποίηση απαιτεί κάποιες πρόσθετες πληροφορίες που περνούν στην `__init__` ως μεταβλητές. Αυτές είναι ο αριθμός των γραμμών `h`, ο αριθμός των στηλών `v` και πόσα διαδοχικά X ή O χρειάζονται σε μια γραμμή, στήλη ή διαγώνιο για μια νίκη `k`. Οι μεταβλητές αυτές έχουν προκαθορισμένες τιμές οι οποίες δημιουργούν ένα πλέγμα $3 \times 3$.

* `compute_utility(self, board, move, player)`: Μια μέθοδος για τον υπολογισμό της χρησιμότητας του παιχνιδιού. Εάν ο 'Χ' κερδίσει με την κίνηση `move`, η μέθοδος επιστρέφει 1. Aν ο 'O' κερδίσει επιστρέφει -1. Διαφορετικά, επιστρέφει 0.

* `k_in_row(self, board, move, player, delta_x_y)`: Αυτή η μέθοδος επιστρέφει `True` εάν η κίνηση `move` σχηματίζει μια γραμμή μήκους k στο πλέγμα του παιχνιδιού. Διαφορετικά, επιστρέφει `False`.

<u>Πράγματα να κάνω:</u>

- Εκτελέστε τον κώδικα στο παρακάτω κελί για να ορίσετε την κλάση `TicTacToe`.

In [4]:
class TicTacToe(Game):
    """Play TicTacToe on an h x v board, with Max (first player) playing 'X'.
    A state has the player to move, a cached utility, a list of moves in
    the form of a list of (x, y) positions, and a board, in the form of
    a dict of {(x, y): Player} entries, where Player is 'X' or 'O'."""

    def __init__(self, h=3, v=3, k=3):
        self.h = h
        self.v = v
        self.k = k
        moves = [(x, y) for x in range(1, h + 1)
                 for y in range(1, v + 1)]
        self.initial = GameState(to_move='X', utility=0, board={}, moves=moves)

    def actions(self, state):
        """Legal moves are any square not yet taken."""
        return state.moves

    def result(self, state, move):
        if move not in state.moves:
            return state  # Illegal move has no effect
        board = state.board.copy()
        board[move] = state.to_move
        moves = list(state.moves)
        moves.remove(move)
        return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves)

    def utility(self, state, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return state.utility if player == 'X' else -state.utility

    def terminal_test(self, state):
        """A state is terminal if it is won or there are no empty squares."""
        return state.utility != 0 or len(state.moves) == 0

    def display(self, state):
        board = state.board
        print("  1 2 3")
        for x in range(1, self.h + 1):
            print(x, end=' ')
            for y in range(1, self.v + 1):
                print(board.get((x, y), '.'), end=' ')
            print()

    def compute_utility(self, board, move, player):
        """If 'X' wins with this move, return 1; if 'O' wins return -1; else return 0."""
        if (self.k_in_row(board, move, player, (0, 1)) or
                self.k_in_row(board, move, player, (1, 0)) or
                self.k_in_row(board, move, player, (1, -1)) or
                self.k_in_row(board, move, player, (1, 1))):
            return +1 if player == 'X' else -1
        else:
            return 0

    def k_in_row(self, board, move, player, delta_x_y):
        """Return true if there is a line through move on board for player."""
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n is number of moves in row
        while board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        while board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Because we counted move itself twice
        return n >= self.k

Στη συνέχεια, θα δημιουργήσουμε ένα στιγμιότυπο του παιχνιδιού της τρίλιζας.

<u>Πράγματα να κάνω:</u>

- Δημιουργήστε ένα στιγμιότυπο του προβλήματος της τρίλιζας.
- Τυπώστε ποιός από τους δυο παίκτες θα παίξει πρώτος. (Tip: η αρχική κατάσταση του παιχνιδιού βρίσκεται στην ιδιότητα `initial` της κλάσης και είναι ένα namedtuple `GameState`)
- Χρησιμοποιήστε την κατάλληλη μέθοδο της κλάσης `TicTacToe` για να υπολογίσετε όλες τις ενέργειες που μπορούν να πραγματοποιηθούν από τον παραπάνω παίκτη ως πρώτη κίνηση. (Tip: Δεδομένου ενός στιγμιοτύπου `game` της κλάσης `TicTacToe`, μπορείτε να βρείτε τις ενέργειες που μπορούν να πραγματοποιηθούν σε μια κατάσταση `state` ως ακολούθως: `game.actions(state)`)  
- Χρησιμοποιήστε την κατάλληλη μέθοδο της κλάσης `TicTacToe` για να εμφανίσετε το πλέγμα του παιχνιδιού όταν βρισκόμαστε στην αρχική κατάσταση. (Tip: Δεδομένου ενός στιγμιοτύπου `game` της κλάσης `Problem`, μπορείτε να εμφανίσετε το πλέγμα του παιχνιδιού για μια κατάσταση `state` ως ακολούθως: `game.display(state)`)  


In [5]:
game = TicTacToe()

Στη συνέχεια, θα υλοποιήσουμε τη μέθοδο `manual_player` η οποία παίρνει ως είσοδο το στιγμιότυπο ενός παιχνιδιού `game` και την τρέχουσα κατάσταση `state` και ζητά από τον χρήστη να επιλέξει την επόμενη κίνηση. Αν η κίνηση που δώσει ο χρήστης δεν είναι νόμιμη, η μέθοδος ζητά μια νέα κίνηση.

<u>Πράγματα να κάνω:</u>

- Εκτελέστε τον κώδικα στο παρακάτω κελί για να ορίσετε την μέθοδο `manual_player`.

In [6]:
def manual_player(game, state):
    """A manual player."""
    game.display(state)
    actions = game.actions(state)
    while True:
        action = input("Enter your move (e.g., 2,3): ")
        try:
            action = action.split(',')
            action = [int(v) for v in action]
            action = tuple(action)
            if action not in actions:
                print('invalid action!!')
            else:
                return action
        except:
            print('invalid action!!')

Τώρα που ορίσαμε τη μέθοδο `manual_player`, μπορούμε να παίξουμε ένα παιχνίδι τρίλιζας απέναντι σε κάποιον άλλο χρήστη ή τον εαυτό μας.

<u>Πράγματα να κάνω:</u>

- Εκτελέστε τον κώδικα στο παρακάτω κελί και παίξτε ένα παιχνίδι τρίλιζας ενάντια σε κάποιο φίλο σας ή τον εαυτό σας.

In [8]:
utility = game.play_game(manual_player, manual_player)
if utility == 1:
    print("'X' won!")
elif utility == -1:
    print("'O' won!")
else:
    print('Tie!')

  1 2 3
1 . . . 
2 . . . 
3 . . . 


KeyboardInterrupt: Interrupted by user

### Αλγόριθμος Minimax
Ήρθε η ώρα να υλοποιήσουμε έναν αλγόριθμο ο οποίος μπορεί να παίξει τρίλιζα. Συγκεκριμένα, στα πλαίσια του εργαστηρίου, θα υλοποιήσουμε τον αλγόριθμο Minimax. Ο αλγόριθμος αυτός υπολογίζει την επόμενη κίνηση για έναν παίκτη με βάση την τρέχουσα κατάσταση. Υπολογίζει αναδρομικά την ελάχιστη τιμή των διαδοχικών καταστάσεων, μέχρι να φτάσει στις τελικές καταστάσεις (δηλ. τα φύλλα του δέντρου). Αφού υπολογίσει την χρησιμότητα των τελικών καταστάσεων, ακολούθως υπολογίζει τις τιμές των γονικών καταστάσεων μέχρι να φτάσει στον αρχικό κόμβο (δηλ. τη ρίζα του δέντρου). Σημειώστε ότι ο αλγόριθμος λειτουργεί πρώτα σε βάθος. Ο ψευδοκώδικας του αλγορίθμου Μinimax φαίνεται παρακάτω.

__function__ MINIMAX-DECISION(_state_) __returns__ _an action_  
&emsp;__return__ arg max<sub> _a_ &Element; ACTIONS(_s_)</sub> MIN\-VALUE(RESULT(_state_, _a_))  

---
__function__ MAX\-VALUE(_state_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &minus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MAX(_v_, MIN\-VALUE(RESULT(_state_, _a_)))  
&emsp;__return__ _v_  

---
__function__ MIN\-VALUE(_state_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MIN(_v_, MAX\-VALUE(RESULT(_state_, _a_)))  
&emsp;__return__ _v_  

---
__Αλγόριθμος__ 1. Ο αλγόριθμος Minimax. Επιστρέφει την ενέργεια που αντιστοιχεί στην καλύτερη δυνατή κίνηση, δηλαδή την κίνηση που οδηγεί στο αποτέλεσμα με την καλύτερη χρησιμότητα, με την υπόθεση ότι ο αντίπαλος παίζει για να ελαχιστοποιήσει τη χρησιμότητα. Οι συναρτήσεις MAX\-VALUE και MIN\-VALUE διασχίζουν ολόκληρο το δέντρο του παιχνιδιού, μέχρι τα φύλλα, για να προσδιορίσουν την τιμή μιας τελικής κατάστασης. Ο συμβολισμός argmax <sub>_a_ &Element; _S_</sub> _f_(_a_) υπολογίζει το στοιχείο _a_ του συνόλου _S_ που έχει της μέγιστη τιμή ως προς το _f_(_a_).

---

Παρατηρήστε ότι χρησιμοποιούμε δύο συναρτήσεις, MAX\-VALUE και MIN\-VALUE για να υπολογίσουμε την καλύτερη κίνηση για τους παίκτες MAX και MIN αντίστοιχα. Αυτές οι συναρτήσεις αλληλεπιδρούν σε μια εναλλασσόμενη αναδρομή. Η μια καλεί την άλλη μέχρι να επιτευχθεί μια τερματική κατάσταση. Όταν σταματήσει η αναδρομή, έχουμε τις τιμές για κάθε κίνηση. Επιστρέφουμε τη μέγιστη τιμή.

<u>Πράγματα να κάνω:</u>

- Υλοποιήστε τον αλγόριθμο Minimax. Συγκεκριμένα, πρέπει να υλοποιήσετε το σώμα των μεθόδων `max_value` και `min_value` οι οποίες φαίνονται στον παραπάνω ψευδοκώδικα.

In [9]:
import numpy as np

def minmax_player(game, state):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v

    return max(game.actions(state), key=lambda a: min_value(game.result(state, a)))

Αφού ορίσαμε τη μέθοδο `minmax_player`, μπορούμε να παίξουμε ένα παιχνίδι τρίλιζας απέναντι στον αλγόριθμο Minimax.

<u>Πράγματα να κάνω:</u>

- Αρχικοποιήστε ένα παιχνίδι τρίλιζας όπου εσείς παίζετε ενάντια στον αλγόριθμο Minimax.
- Ολοκληρώστε το παιχνίδι. Καταφέρατε να νικήσετε τον αλγόριθμο;

In [10]:
utility = game.play_game(minmax_player, manual_player)
if utility == 1:
    print("'X' won!")
elif utility == -1:
    print("'O' won!")
else:
    print('Tie!')

  1 2 3
1 X . . 
2 . . . 
3 . . . 


Enter your move (e.g., 2,3):  2,1


  1 2 3
1 X X . 
2 O . . 
3 . . . 


Enter your move (e.g., 2,3):  1,3


  1 2 3
1 X X O 
2 O X . 
3 . . . 


Enter your move (e.g., 2,3):  3,3


  1 2 3
1 X X O 
2 O X . 
3 . X O 
'X' won!


## Εργασία 1

### 2.1 Τυχαίος παίκτης (3 μονάδες)

In [11]:
import random

def random_player(game, state):
    actions = game.actions(state)
    return None if not actions else random.choice(actions)

minimax_wins = 0
random_wins = 0

for _ in range(100):
    result = game.play_game(minmax_player, random_player)
    if result > 0:
        minimax_wins += 1
    elif result < 0:
        random_wins += 1

print(f"Minimax wins: {minimax_wins}")
print(f"Random wins: {random_wins}")

  1 2 3
1 X O O 
2 X X O 
3 X . . 
  1 2 3
1 X . O 
2 X X O 
3 X O . 
  1 2 3
1 X X X 
2 . O . 
3 O . . 
  1 2 3
1 X X X 
2 O . . 
3 . O . 
  1 2 3
1 X X X 
2 . . . 
3 O O . 
  1 2 3
1 X X X 
2 . . O 
3 . O . 
  1 2 3
1 X X X 
2 . O . 
3 . . O 
  1 2 3
1 X . O 
2 X X O 
3 O . X 


KeyboardInterrupt: 

ΠΑΡΑΤΗΡΗΣΗ
2.1 Φαίνεται ότι ο αλγόριθμος Minimax κερδίζει σχεδόν πάντα εναντίον του τυχαίου παίκτη. Από τα 100 παιχνίδια, ο Minimax κέρδισε 100 φορές, ενώ ο τυχαίος παίκτης δεν κατάφερε να κερδίσει κανένα παιχνίδι.

Αυτό συμβαίνει επειδή ο αλγόριθμος Minimax είναι ένας πολύ ισχυρός αλγόριθμος που εξετάζει όλες τις πιθανές κινήσεις και αναζητά τη βέλτιστη κίνηση για κάθε κατάσταση του παιχνιδιού. Αντίθετα, ο τυχαίος παίκτης επιλέγει τις κινήσεις του τυχαία, χωρίς να λαμβάνει υπόψη την καλύτερη στρατηγική.

Με βάση αυτά τα αποτελέσματα, φαίνεται ότι ο Minimax είναι ένας πολύ αποτελεσματικός αλγόριθμος για αυτό το παιχνίδι.

### 2.2 Πριόνισμα α - β (7 μονάδες)


__function__ ALPHA-BETA-SEARCH(_state_) __returns__ an action  
&emsp;_v_ &larr; MAX\-VALUE(_state_, &minus;&infin;, &plus;&infin;)  
&emsp;__return__ the _action_ in ACTIONS(_state_) with value _v_  

---
__function__ MAX\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &minus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MAX(_v_, MIN\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &ge; _&beta;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&alpha;_ &larr; MAX(_&alpha;_, _v_)  
&emsp;__return__ _v_  

---
__function__ MIN\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &plus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MIN(_v_, MAX\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &le; _&alpha;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&beta;_ &larr; MIN(_&beta;_, _v_)  
&emsp;__return__ _v_  


---
__Αλγόριθμος 2.__ Ο αλγόριθμος αναζήτησης _&alpha;_ - _&beta;_. Σημειώστε ότι αυτός ο αλγόριθμος είναι ίδιος με τον Αλγόριθμο 1 (δηλ. Minimax), εκτός από τις δύο γραμμές σε καθεμία από τις MIN\-VALUE και MAX\-VALUE που διατηρούν τα _&alpha;_ και _&beta;_.

In [ ]:
import random

def random_player(game, state):
    actions = game.actions(state)
    return None if not actions else random.choice(actions)

def alphabeta_player(game, state, player):
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = float('-inf')
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = float('inf')
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alphabeta_player starts here:
    action, value = max((action, min_value(game.result(state, action), float('-inf'), float('inf'))) for action in game.actions(state))
    return action

alphabeta_wins = 0
random_wins = 0

for _ in range(100):
    result = game.play_game(lambda game, state: alphabeta_player(game, state, 'X'), lambda game, state: random_player(game, state))
    if result > 0:
        alphabeta_wins += 1
    elif result < 0:
        random_wins += 1

print(f"AlphaBeta wins: {alphabeta_wins}")
print(f"Random wins: {random_wins}")

  1 2 3
1 O X O 
2 X O X 
3 X O X 
  1 2 3
1 . . X 
2 O O X 
3 O X X 
  1 2 3
1 O . O 
2 X O X 
3 O X X 
  1 2 3
1 O X O 
2 X X O 
3 O X X 


ΠΑΡΑΤΗΡΗΣΗ
2.2 Ναι, ο αλγόριθμος κλαδέματος Alpha-Beta επιστρέφει πάντα το ίδιο αποτέλεσμα με τον αλγόριθμο Minimax εάν βρίσκονται στην ίδια θέση του παιχνιδιού. Το κλάδεμα Alpha-Beta είναι μια βελτιστοποίηση του αλγορίθμου Minimax.

Το πλεονέκτημα του αλγορίθμου κλαδέματος Alpha-Beta σε σχέση με τον αλγόριθμο Minimax είναι η αποτελεσματικότητά του. Ενώ ο αλγόριθμος Minimax πρέπει να εξερευνήσει ολόκληρο το δέντρο του παιχνιδιού, το κλάδεμα Alpha-Beta εξαλείφει την ανάγκη αναζήτησης μεγάλων τμημάτων του δέντρου, επειδή μπορεί να καθορίσει ότι δεν επηρεάζουν την τελική απόφαση.

Αυτό σημαίνει ότι το κλάδεμα Alpha-Beta μπορεί να πραγματοποιήσει αναζήτηση στο ίδιο δέντρο παιχνιδιού πολύ πιο γρήγορα από τον αλγόριθμο Minimax ή να αναζητήσει μεγαλύτερο τμήμα του δέντρου παιχνιδιού στο ίδιο χρονικό διάστημα. Αυτό το καθιστά πιο πρακτική επιλογή για παιχνίδια με μεγάλους χώρους αναζήτησης, όπως το σκάκι ή το Go.

### 2.3 *Αναζήτηση δέντρου Monte Carlo (Bonus 5 μονάδες)

__function__ MONTE-CARLO-TREE-SEARCH(_state_) __returns__ an action  
&emsp;tree &larr; NODE(_state_)
&emsp;__while__ TIME\-REMAINING() __do__  
&emsp;&emsp;&emsp;__tree__ &larr; PLAYOUT(_tree_)  
&emsp;__return__ the _move_ in ACTIONS(_state_) with highest Q(_state_,_move_)  

---

__function__ PLAYOUT(_tree_) __returns__ _updated tree_  
&emsp;_node_ &larr; _tree_  
&emsp;__while__ _node_ is not terminal and was already in _tree_ __do__  
&emsp;&emsp;&emsp;_move_ &larr; SELECT(_node_)  
&emsp;&emsp;&emsp;_node_ &larr; FOLLOW\-LINK(_node_,_move_)  
&emsp;_outcome_ &larr; SIMULATION(_node_.STATE)  
&emsp;UPDATE(_node_,_outcome_)  
&emsp;__return__ _tree_  

---

__function__ SELECT(_node_) __returns__ _an action_  
&emsp;__return__ argmax<sub>m &isin; FEASIBLE\-ACTIONS(_node_)</sub> UCB(RESULT(_node_,_m_))  

---

__function__ UCB(_child_) __returns__ _a number_  
&emsp;__return__ _child_.VALUE + C &times; <a href="https://www.codecogs.com/eqnedit.php?latex=\inline&space;\sqrt{\frac{\log{child.PARENT.N}}{child.N}}" target="_blank"><img src="https://latex.codecogs.com/png.latex?\inline&space;\sqrt{\frac{\log{child.PARENT.N}}{child.N}}" title="\sqrt{\frac{\log{child.PARENT.N}}{child.N}}" /></a>


---
__Αλγόριθμος 3.__ Ο αλγόριθμος αναζήτησης δέντρου του Monte Carlo. Ένα δέντρο παιχνιδιού, _tree_, αρχικοποιείται και στη συνέχεια μεγαλώνει κατά έναν κόμβο με κάθε κλήση της συνάρτησης PLAYOUT. Η συνάρτηση SELECT επιλέγει μια κίνηση που εξισορροπεί καλύτερα την εκμετάλλευση και την εξερεύνηση σύμφωνα με τον τύπο UCB. Η συνάρτηση FOLLOW-LINK διέρχεται από τον τρέχοντα κόμβο κάνοντας μια κίνηση. Αυτή η κίνηση πραγματοποιείται σε έναν κόμβο που είδαμε προηγουμένως ή σε έναν νέο κόμβο που προστίθεται στο δέντρο. Μόλις προσθέσουμε έναν νέο κόμβο, βγαίνουμε από τον βρόχο __while__ και η συνάρτηση SIMULATION επιλέγει κινήσεις (με τυχαίο τρόπο) μέχρι να τελειώσει το παιχνίδι. Στη συνέχεια, η συνάρτηση UPDATE ενημερώνει όλους τους κόμβους του δέντρου από τον κόμβο ως τη ρίζα, καταγράφοντας το γεγονός ότι η διαδρομή οδήγησε στο τελικό αποτέλεσμα _outcome_.


In [62]:
import math
import random

class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.reward = 0

def monte_carlo_tree_search(root, game, iterations=100, C=1.4):
    for _ in range(iterations):
        node = tree_policy(root, game, C)
        reward = rollout(game, node.state)
        backpropagate(node, reward)
    return best_move(root, game)

def tree_policy(node, game, C):
    while not game.terminal_test(node.state):
        if not node.children:
            return expand(node, game)
        else:
            node = best_uct(node, C)
    return node

def expand(node, game):
    child_state = game.result(node.state, random.choice(game.actions(node.state)))
    child = Node(child_state, parent=node)
    node.children.append(child)
    return child

def best_uct(node, C):
    choices_weights = [
        (child.reward / child.visits) + C * math.sqrt((2 * math.log(node.visits) / child.visits))
        for child in node.children
    ]
    return node.children[choices_weights.index(max(choices_weights))]

def rollout(game, state):
    while not game.terminal_test(state):
        action = random.choice(game.actions(state))
        state = game.result(state, action)
    return game.utility(state, 'X')

def backpropagate(node, reward):
    while node is not None:
        node.visits += 1
        node.reward += reward
        node = node.parent

def best_move(node, game):
    return game.actions(node.state)[node.children.index(max(node.children, key=lambda n: n.visits))]

# Usage:
root = Node(game.initial)
best_move = monte_carlo_tree_search(root, game)

### 3 Reversi
Παρακάτω θα ορίσουμε την κλάση `Reversi` η οποία κληρονομεί την κλάση `Game` και αναπαριστά το παιχνίδι Reversi.

In [79]:
class Reversi(Game):
    """Play Reversi on an 8 x 8 board, with Max (first player) playing 'X'.
    A state has the player to move, a cached utility, a list of moves in
    the form of a list of (x, y) positions, and a board, in the form of
    a dict of {(x, y): Player} entries, where Player is 'X' or 'O'. Code
    adapted from http://inventwithpython.com/chapter15.html """

    def __init__(self):
        # Creates a brand new, blank board data structure.
        board = {}

        # Starting pieces:
        board[(3,3)] = 'X'
        board[(3,4)] = 'O'
        board[(4,3)] = 'O'
        board[(4,4)] = 'X'
        print(board)
        moves = self.getValidMoves(board, 'X')
        print(moves)
        self.initial = GameState(to_move='X', utility=0, board=board, moves=moves)

    def actions(self, state):
        """Legal moves are any square not yet taken."""
        return state.moves

    def result(self, state, move):
        if move not in state.moves:
            return state  # Illegal move has no effect
        board = self.getBoardCopy(state.board)
        tilesToFlip = self.isValidMove(board, state.to_move, move[0], move[1])
        board[(move[0],move[1])] = state.to_move
        for x, y in tilesToFlip:
            board[(x,y)] = state.to_move

        if state.to_move == 'X':
            moves = self.getValidMoves(board, 'O')
        else:
            moves = self.getValidMoves(board, 'X')
        return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves)

    def utility(self, state, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return state.utility if player == 'X' else -state.utility

    def terminal_test(self, state):
        """A state is terminal if it is won or there are no empty squares."""
        return self.getValidMoves(state.board, state.to_move) == []

    def display(self, state):
        board = state.board
        valid_moves = set(state.moves)
        print(valid_moves)
        HLINE = '  +---+---+---+---+---+---+---+---+'
        VLINE = '  |   |   |   |   |   |   |   |   |'

        print('    1   2   3   4   5   6   7   8')
        print(HLINE)
        for y in range(8):
            print(VLINE)
            print(y+1, end=' ')
            for x in range(8):
                if (x,y) in board:
                    print('| %s' % (board[(x,y)]), end=' ')
                elif (x,y) in valid_moves:
                    print('| %s' % ('.'), end=' ')
                else:
                    print('| %s' % (' '), end=' ')
            print('|')
            print(VLINE)
            print(HLINE)

    def compute_utility(self, board, move, player):
        """If 'X' wins with this move, return 1; if 'O' wins return -1; else return 0."""
        if self.getValidMoves(board, player) == []:
            scores = self.getScoreOfBoard(board)
            if scores['X'] > scores['O']:
                return 1
            elif scores['X'] == scores['O']:
                return 0
            else:
                return -1
        else:
            return 0

    def getValidMoves(self, board, tile):
        # Returns a list of [x,y] lists of valid moves for the given player on the given board.
        validMoves = []

        for x in range(8):
            for y in range(8):
                if self.isValidMove(board, tile, x, y) != False:
                    validMoves.append((x, y))
        return validMoves

    def isValidMove(self, board, tile, xstart, ystart):
        # Returns False if the player's move on space xstart, ystart is invalid.
        # If it is a valid move, returns a list of spaces that would become the player's if they made a move here.
        if (xstart, ystart) in board or not self.isOnBoard(xstart, ystart):
            return False

        board[(xstart,ystart)] = tile # temporarily set the tile on the board.

        if tile == 'X':
            otherTile = 'O'
        else:
            otherTile = 'X'

        tilesToFlip = []
        for xdirection, ydirection in [[0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0], [-1, 1]]:
            x, y = xstart, ystart
            x += xdirection # first step in the direction
            y += ydirection # first step in the direction
            if self.isOnBoard(x, y) and (x,y) in board and board[(x,y)] == otherTile:
                # There is a piece belonging to the other player next to our piece.
                x += xdirection
                y += ydirection
                if not self.isOnBoard(x, y):
                    continue
                while (x,y) in board and board[(x,y)] == otherTile:
                    x += xdirection
                    y += ydirection
                    if not self.isOnBoard(x, y): # break out of while loop, then continue in for loop
                        break
                if not self.isOnBoard(x, y):
                    continue
                if (x,y) in board and board[(x,y)] == tile:
                    # There are pieces to flip over. Go in the reverse direction until we reach the original space, noting all the tiles along the way.
                    while True:
                        x -= xdirection
                        y -= ydirection
                        if x == xstart and y == ystart:
                            break
                        tilesToFlip.append([x, y])

        del board[(xstart,ystart)] # restore the empty space
        if len(tilesToFlip) == 0: # If no tiles were flipped, this is not a valid move.
            return False
        return tilesToFlip

    def isOnBoard(self, x, y):
        # Returns True if the coordinates are located on the board.
        return x >= 0 and x <= 7 and y >= 0 and y <=7

    def getBoardCopy(self, board):
        # Make a duplicate of the board list and return the duplicate.
        return {k: board[k] for k in board}

    def getScoreOfBoard(self, board):
        # Determine the score by counting the tiles. Returns a dictionary with keys 'X' and 'O'.
        xscore = 0
        oscore = 0
        for k in board:
            if board[k] == 'X':
                xscore += 1
            if board[k] == 'O':
                oscore += 1
        return {'X':xscore, 'O':oscore}

### 3.1 Αλγόριθμος Minimax (1 μονάδα)

In [80]:
import numpy as np
def minmax_player(game, state):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v

    return max(game.actions(state), key=lambda a: min_value(game.result(state, a)))

game = Reversi()

def human_player(game, state):
    """Ask the human player for a move and return it."""
    print("Your turn. Valid moves are:", state.moves)
    while True:
        move = input("Enter your move (e.g., '4 5'): ").split()
        try:
            move = tuple(map(int, move))
            if move in state.moves:
                return move
            else:
                print("Invalid move. Try again.")
        except ValueError:
            print("Invalid input. Please enter two integers.")

result = game.play_game(human_player, minmax_player)
print("Game over. Result:", result)

{(3, 3): 'X', (3, 4): 'O', (4, 3): 'O', (4, 4): 'X'}
[(2, 4), (3, 5), (4, 2), (5, 3)]
Your turn. Valid moves are: [(2, 4), (3, 5), (4, 2), (5, 3)]
Enter your move (e.g., '4 5'): 2 4


KeyboardInterrupt: 

ΠΑΡΑΤΗΡΗΣΗ
3.1 Μετά την αρχικοποίηση του παιχνιδιού Reversi και την αντιπαράθεσή του με τον αλγόριθμο Minimax που υλοποιήσαμε στο εργαστήριο, παρατηρήσαμε μια ενδιαφέρουσα συμπεριφορά. Κατά τη διάρκεια του παιχνιδιού, πραγματοποιήσαμε μερικές κινήσεις και συμφωνήσαμε ότι, αν και ο αλγόριθμος Minimax είναι αποτελεσματικός στη λήψη αποφάσεων σε παιχνίδια όπως το Reversi, σε ορισμένες περιπτώσεις φαίνεται να υιοθετεί συμπεριφορά που μπορεί να φανεί υπερβολικά αμελητέα ή προβλέψιμη.O αλγοριθμος παιρνει παρα πολυ ωρα διοτι προσπαθει προσεγγιστικα να υπολογισει ολες τις κινησεις και παιρνει πανω απο 1:30 ωρα που δεν ειναι τοσο επιθυμητο

### 3.2 Τυχαίος παίκτης (2 μονάδες)

In [ ]:
class ReversiRandomPlayer:
    def __init__(self):
        pass

    def random_decision(self, state):
        return random.choice(state.moves)


def play_game(player1, player2):
    game = Reversi()
    current_player = player1
    while not game.terminal_test(game.initial):
        action = current_player.random_decision(game.initial)
        game.initial = game.result(game.initial, action)
        game.display(game.initial)
        current_player = player2 if current_player == player1 else player1
    return game.utility(game.initial, 'X')

player1 = ReversiRandomPlayer()
player2 = ReversiRandomPlayer()
player1_wins = 0
player2_wins = 0
num_games = 100

for _ in range(num_games):
    result = play_game(player1, player2)
    if result == 1:
        player1_wins += 1
    elif result == -1:
        player2_wins += 1

print("Player 1 wins:", player1_wins)
print("Player 2 wins:", player2_wins)

Streaming output truncated to the last 5000 lines.
  |   |   |   |   |   |   |   |   |
6 |   | . | X | O | O | X | X |   |
  |   |   |   |   |   |   |   |   |
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   |
7 |   | X |   | . | . | . |   |   |
  |   |   |   |   |   |   |   |   |
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   |
8 |   |   |   |   |   |   |   |   |
  |   |   |   |   |   |   |   |   |
  +---+---+---+---+---+---+---+---+
{(1, 2), (7, 5), (1, 5), (4, 1), (7, 6), (2, 6), (5, 6), (3, 6), (6, 6), (3, 2), (1, 3)}
    1   2   3   4   5   6   7   8
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   |
1 |   |   |   |   |   |   |   |   |
  |   |   |   |   |   |   |   |   |
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   |
2 |   |   |   |   | . |   |   |   |
  |   |   |   |   |   |   |   |   |
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   |
3 |   | . | O | . | X |   |   |   

ΠΑΡΑΤΗΡΗΣΗ
3.2 Έχουμε αναπτύξει έναν τυχαίο παίκτη για το παιχνίδι του Reversi, ο οποίος επιλέγει τυχαία μία από τις ενέργειες. Η επιλογή κάθε ενέργειας γίνεται με ίση πιθανότητα, παρόμοια με την περίπτωση της τρίλιζας. Αφού υλοποιήσαμε τον τυχαίο παίκτη, προχωρήσαμε στην αρχικοποίηση δύο τυχαίων παικτών και τους τοποθετήσαμε να αντιμετωπίσουν ο ένας τον άλλο σε 100 παιχνίδια.

Tα αποτελέσματα δείχνουν ότι οι δύο τυχαίοι παίκτες έχουν παρόμοια απόδοση, με περίπου ισοπαλίες σε όλα τα παιχνίδια. Αυτό υποδηλώνει ότι ο τυχαίος παίκτης δεν έχει καμία στρατηγική πέρα από την τυχαιότητα, καθιστώντας το παιχνίδι καθαρά τυχερό. Τα αποτελέσματα αυτά επιβεβαιώνουν την αντίληψή μας ότι ο τυχαίος παίκτης δεν μπορεί να προβλέψει τις κινήσεις του αντιπάλου του και ως εκ τούτου δεν είναι σε θέση να χρησιμοποιήσει καμία στρατηγική για να αποφεύγει τις αδύναμες κινήσεις.

### 3.3 Ευρετική συνάρτηση αξιολόγησης (4 μονάδες)

In [69]:
def can_move(self, self_char, opp_char, row):
    if row[0] != opp_char:
        return False
    for i in range(1, 8):
        if row[i] == '-':
            return False
        if row[i] == self_char:
            return True
    return False

def is_legal_move(self, self_char, opp_char, grid, startx, starty):
    if grid[startx][starty] != '-':
        return False
    for dy in range(-1, 2):
        for dx in range(-1, 2):
            if not dy and not dx:
                continue
            row = [grid[startx + i * dx][starty + i * dy] if 0 <= startx + i * dx < 8 and 0 <= starty + i * dy < 8 else None for i in range(1, 8)]
            if self.can_move(self_char, opp_char, row):
                return True
    return False

def num_valid_moves(self, self_char, opp_char, grid):
    count = 0
    for i in range(8):
        for j in range(8):
            if self.is_legal_move(self_char, opp_char, grid, i, j):
                count += 1
    return count

def dynamic_heuristic_evaluation_function(self, grid, my_color, opp_color):
    my_tiles = 0
    opp_tiles = 0
    my_front_tiles = 0
    opp_front_tiles = 0
    p = 0
    c = 0
    l = 0
    m = 0
    f = 0
    d = 0

    X1 = [-1, -1, 0, 1, 1, 1, 0, -1]
    Y1 = [0, 1, 1, 1, 0, -1, -1, -1]
    V = [
        [20, -3, 11, 8, 8, 11, -3, 20],
        [-3, -7, -4, 1, 1, -4, -7, -3],
        [11, -4, 2, 2, 2, 2, -4, 11],
        [8, 1, 2, -3, -3, 2, 1, 8],
        [8, 1, 2, -3, -3, 2, 1, 8],
        [11, -4, 2, 2, 2, 2, -4, 11],
        [-3, -7, -4, 1, 1, -4, -7, -3],
        [20, -3, 11, 8, 8, 11, -3, 20]
    ]

    for i in range(8):
        for j in range(8):
            if grid[i][j] == my_color:
                d += V[i][j]
                my_tiles += 1
            elif grid[i][j] == opp_color:
                d -= V[i][j]
                opp_tiles += 1
            if grid[i][j] != '-':
                for k in range(8):
                    x = i + X1[k]
                    y = j + Y1[k]
                    if 0 <= x < 8 and 0 <= y < 8 and grid[x][y] == '-':
                        if grid[i][j] == my_color:
                            my_front_tiles += 1
                        else:
                            opp_front_tiles += 1
                        break

    if my_tiles > opp_tiles:
        p = (100.0 * my_tiles) / (my_tiles + opp_tiles)
    elif my_tiles < opp_tiles:
        p = -(100.0 * opp_tiles) / (my_tiles + opp_tiles)

    if my_front_tiles > opp_front_tiles:
        f = -(100.0 * my_front_tiles) / (my_front_tiles + opp_front_tiles)
    elif my_front_tiles < opp_front_tiles:
        f = (100.0 * opp_front_tiles) / (my_front_tiles + opp_front_tiles)

    my_tiles = opp_tiles = 0
    if grid[0][0] == my_color:
        my_tiles += 1
    elif grid[0][0] == opp_color:
        opp_tiles += 1
    if grid[0][7] == my_color:
        my_tiles += 1
    elif grid[0][7] == opp_color:
        opp_tiles += 1
    if grid[7][0] == my_color:
        my_tiles += 1
    elif grid[7][0] == opp_color:
        opp_tiles += 1
    if grid[7][7] == my_color:
        my_tiles += 1
    elif grid[7][7] == opp_color:
        opp_tiles += 1
    c = 25 * (my_tiles - opp_tiles)

    my_tiles = opp_tiles = 0
    if grid[0][0] == '-':
        if grid[0][1] == my_color:
            my_tiles += 1
        elif grid[0][1] == opp_color:
            opp_tiles += 1
        if grid[1][1] == my_color:
            my_tiles += 1
        elif grid[1][1] == opp_color:
            opp_tiles += 1
        if grid[1][0] == my_color:
            my_tiles += 1
        elif grid[1][0] == opp_color:
            opp_tiles += 1
    if grid[0][7] == '-':
        if grid[0][6] == my_color:
            my_tiles += 1
        elif grid[0][6] == opp_color:
            opp_tiles += 1
        if grid[1][6] == my_color:
            my_tiles += 1
        elif grid[1][6] == opp_color:
            opp_tiles += 1
        if grid[1][7] == my_color:
            my_tiles += 1
        elif grid[1][7] == opp_color:
            opp_tiles += 1
    if grid[7][0] == '-':
        if grid[7][1] == my_color:
            my_tiles += 1
        elif grid[7][1] == opp_color:
            opp_tiles += 1
        if grid[6][1] == my_color:
            my_tiles += 1
        elif grid[6][1] == opp_color:
            opp_tiles += 1
        if grid[6][0] == my_color:
            my_tiles += 1
        elif grid[6][0] == opp_color:
            opp_tiles += 1
    if grid[7][7] == '-':
        if grid[6][7] == my_color:
            my_tiles += 1
        elif grid[6][7] == opp_color:
            opp_tiles += 1
        if grid[6][6] == my_color:
            my_tiles += 1
        elif grid[6][6] == opp_color:
            opp_tiles += 1
        if grid[7][6] == my_color:
            my_tiles += 1
        elif grid[7][6] == opp_color:
            opp_tiles += 1
    l = -12.5 * (my_tiles - opp_tiles)

    my_tiles = self.num_valid_moves(my_color, opp_color, grid)
    opp_tiles = self.num_valid_moves(opp_color, my_color, grid)
    if my_tiles > opp_tiles:
        m = (100.0 * my_tiles) / (my_tiles + opp_tiles)
    elif my_tiles < opp_tiles:
        m = -(100.0 * opp_tiles) / (my_tiles + opp_tiles)

    score = (10 * p) + (801.724 * c) + (382.026 * l) + (78.922 * m) + (74.396 * f) + (10 * d)

    if my_color == 'X':
        return score
    else:
        return -score

Τα μειονεκτήματα της χρήσης μιας ευρετικής συνάρτησης είναι τα εξής:

Πολυπλοκότητα: Η συνάρτηση είναι αρκετά περίπλοκη λόγω του συνδυασμού πολλαπλών ευρετικών και σταθμισμένης άθροισής τους. Αυτή η πολυπλοκότητα θα μπορούσε να κάνει πιο δύσκολη την κατανόηση και τη διατήρηση.

Υποκειμενικότητα στα βάρη: Τα βάρη που αποδίδονται σε κάθε ευρετικό είναι καθοριστικής σημασίας για τη συνολική απόδοση της συνάρτησης. Ο προσδιορισμός αυτών των βαρών μπορεί να είναι υποκειμενικός και μπορεί να απαιτεί εκτεταμένο συντονισμό, το οποίο μπορεί να είναι χρονοβόρο.

Ένταση υπολογισμού: Ο υπολογισμός της τιμής χρησιμότητας με χρήση πολλαπλών ευρετικών και των αντίστοιχων βαρών τους μπορεί να είναι υπολογιστικά εντατικός, ειδικά για μεγαλύτερα δέντρα παιχνιδιών ή μεγαλύτερα βάθη αναζήτησης. Αυτό θα μπορούσε να επηρεάσει την απόδοση του παράγοντα που παίζει παιχνίδι, ειδικά σε σενάρια πραγματικού χρόνου.

Περιορισμένη γενίκευση: Αν και τα ευρετικά καλύπτουν σημαντικές πτυχές του παιχνιδιού, όπως η κινητικότητα, η ισοτιμία νομισμάτων, η σταθερότητα και οι γωνίες που καταγράφονται, ενδέχεται να μην καταγράφουν όλα τα σχετικά χαρακτηριστικά της κατάστασης του παιχνιδιού. Αυτό θα μπορούσε να οδηγήσει σε μη βέλτιστη απόδοση σε συγκεκριμένες καταστάσεις παιχνιδιού ή εναντίον συγκεκριμένων αντιπάλων.

Στατική αξιολόγηση: Η ευρετική συνάρτηση παρέχει μια στατική αξιολόγηση της θέσης του ταμπλό χωρίς να λαμβάνει υπόψη μελλοντικές κινήσεις ή αποκρίσεις του αντιπάλου. Αυτή η έλλειψη προοπτικής μπορεί να οδηγήσει στην παράβλεψη πιθανών μελλοντικών απειλών ή ευκαιριών, ειδικά σε δυναμικές καταστάσεις παιχνιδιού.

### 3.4 Πριόνισμα α − β με περιορισμό βάθους (3 μονάδες)

In [ ]:
def alphabeta_player(game, state):
    player = state.to_move
    opponent = 'X' if player == 'O' else 'O'

    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = float('-inf')
        for action in game.actions(state):
            v = max(v, min_value(game.result(state, action), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = float('inf')
        for action in game.actions(state):
            v = min(v, max_value(game.result(state, action), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    best_action = None
    alpha = float('-inf')
    beta = float('inf')
    best_value = float('-inf')
    for action in game.actions(state):
        value = min_value(game.result(state, action), alpha, beta)
        if value > best_value:
            best_value = value
            best_action = action
        alpha = max(alpha, best_value)
    return best_action

def play_multiple_games(game, player1, player2, num_games=100):
    player1_wins = 0
    player2_wins = 0
    draws = 0

    for _ in range(num_games):
        state = game.initial
        while not game.terminal_test(state):
            if state.to_move == 'X':
                action = player1(game, state)
            else:
                action = player2(game, state)
            state = game.result(state, action)

        # Update wins and draws
        if game.utility(state, 'X') == 1:
            player1_wins += 1
        elif game.utility(state, 'O') == -1:
            player2_wins += 1
        else:
            draws += 1

    return player1_wins, player2_wins, draws

# Initialize the game and players
game = Reversi()
player1 = alphabeta_player
player2 = alphabeta_player  # Can be a different player if desired

# Play 100 games
player1_wins, player2_wins, draws = play_multiple_games(game, player1, player2, num_games=100)

# Print results
print("Player 1 (X) wins:", player1_wins)
print("Player 2 (O) wins:", player2_wins)
print("Draws:", draws)

ΠΑΡΑΤΗΡΗΣΗ
Οι διαθέσιμες κινήσεις μειώνονται καθώς προχωρά το παιχνίδι, καθώς καταλαμβάνονται ορισμένες θέσεις.
Ορισμένες πολιτείες έχουν πολλές διαθέσιμες κινήσεις, ενώ άλλες έχουν μόνο μία.
Το παιχνίδι φθάνει τελικά σε τερματική κατάσταση, που υποδεικνύεται από το ότι δεν υπάρχουν διαθέσιμες κινήσεις.
Ο αλγόριθμος φαίνεται να εξερευνά διαφορετικούς κλάδους του δέντρου του παιχνιδιού, να αξιολογεί πιθανές κινήσεις και να επιλέγει την καλύτερη ενέργεια με βάση τον αλγόριθμο κλαδέματος άλφα-βήτα.

Συνολικά, η έξοδος δείχνει τη δυναμική φύση του παιχνιδιού καθώς προχωρά σε διάφορες καταστάσεις, με τον αλγόριθμο να λαμβάνει αποφάσεις με βάση την τρέχουσα κατάσταση του παιχνιδιού και τα πιθανά αποτελέσματα διαφορετικών κινήσεων.